# Exercises - Genetic Algorithms

In [104]:
from math import log2, ceil, e, cos, pi

In [2]:
from random import randint,random,sample,uniform

In [3]:
from statistics import mean

In [4]:
import numpy as np

In [6]:
sample([2,3,52,5],4)

[52, 5, 3, 2]

In [7]:
def f(x1,x2):
    if not(-4.5 <= x1 <= 4.5 and -4.5 <= x2 <= 4.5):
        return 'bad input: out of domain'
    return (1.5-x1+x1*x2)**2 + (2.25-x1+x1*(x2**2))**2 + (2.625-x1+x1*(x2**3))**2

El mínimo global está en x = (3, 0.5) y f(x) = 0. \*global minimum

In [8]:
f(3,0.5), f(-4.5,-4.5)

(0.0, 181853.61328125)

## 1.
Implementar en el lenguaje de su preferencia, el AG que se diseñó en el ejemplo anterior para resolver la función de Beale.

In [9]:
int('0111010000100011010101011001', 2)

121779545

In [10]:
def bits_req(x_bounds):
    return [ceil(log2((x[1]-(x[0]))*(10**3))) for x in x_bounds]

In [11]:
bits_req([(-4.5,4.5),(-4.5,4.5)])

[14, 14]

In [12]:
def real_from_bits(bit_string, n_variable_bits, n_variable_bounds):
    reals = []
    index = 0
    for n_bits, bound in zip(n_variable_bits,n_variable_bounds):
        bits = bit_string[index:index+n_bits]
        index = n_bits
        d = int(bits, 2)
        r = bound[0] + ((bound[1]-bound[0])*d) / ((2**n_bits)-1)
        reals.append(r)
    return reals

In [13]:
var_bounds = [(-4.5,4.5),(-4.5,4.5)]
n_variable_bits = bits_req(var_bounds)
n_variable_bits

[14, 14]

In [14]:
real_from_bits('11011000000000'+'10001110000000', n_variable_bits, var_bounds)

[3.0942135140084233, 0.49249221754257455]

In [15]:
real_from_bits('01110100001000'+'11010101011001', n_variable_bits, var_bounds)

[-0.41723127632301793, 3.0024720747115916]

In [17]:
example_individual = ('1111111111111111111111011001', n_variable_bits, var_bounds)
example_individual

('1111111111111111111111011001', [14, 14], [(-4.5, 4.5), (-4.5, 4.5)])

In [18]:
def Beale_fitness(bit_string, n_variable_bits_req, n_variable_bounds): # a low value is a bad fitness
    return 1/(f(*real_from_bits(bit_string, n_variable_bits_req, n_variable_bounds))+1)

In [19]:
Beale_fitness(*example_individual)

5.880385373883209e-06

In [20]:
Beale_fitness('11011000000000'+'10001110000000', n_variable_bits, var_bounds)

0.9768962222434674

In [21]:
def random_bin_string(bits):
    string = ''
    for i in range(bits):
        string += str(randint(0,1))
    return string

In [18]:
random_bin_string(4)

'1100'

In [65]:
def random_solution(n_variable_bits_req, n_variable_bounds=False, fitness_func=Beale_fitness):
    individual = ''
    for bits in n_variable_bits_req:
        x_bits = random_bin_string(bits)
        individual += (x_bits)
    if n_variable_bounds:
        return individual, fitness_func(individual, n_variable_bits_req, n_variable_bounds)
    return individual

In [66]:
b_required = bits_req([(2,3),(5,6),(-50,11)])
rand_sol = random_solution(b_required)

b_required, rand_sol, sum(b_required), len(rand_sol)

([10, 10, 16], '000000111000100100111000100111111111', 36, 36)

In [67]:
random_solution(bits_req([(-4.5,4.5),(-4.5,4.5)]))#,n_variable_bounds=[(-4.5,4.5),(-4.5,4.5)])

'1010101011101001111000110000'

In [60]:
random_solution(bits_req([(-4.5,4.5),(-4.5,4.5)]),n_variable_bounds=[(-4.5,4.5),(-4.5,4.5)])

('0101110001101101000000100000', 0.002812569703105383)

In [23]:
def universal_stochastic_selection(population, n_variable_bits_req, n_variable_bounds):
    fitnesses = [x[1] for x in population]
    strings = [x[0] for x in population]
    fitness_mean = mean(fitnesses)
    Ei_s = [fit/fitness_mean for fit in fitnesses]
    r = random()
    parents = []
    sum_ = 0
    for individual,Ei,fit in zip(strings,Ei_s,fitnesses):
        sum_ += Ei
        while sum_ > r:
            parents.append((individual,fit))
            r += 1
    return parents

In [24]:
universal_stochastic_selection([('100010110011110110011000011100110000', 0.15029431701487947),
                                ('110000100011001001100110010010011010', 0.011398346494599433),
                                ('101110111100100000001000011010001010', 2.9777439040564625e-05),
                                ('110101000010111111010100100101110110', 0.008058140115587174),
                                ('001101101111011010101101010101110010', 0.00025309048960844106)],  
                               bits_req([(-4.5, 4.5),(-4.5, 4.5)]),
                               [(-4.5, 4.5),(-4.5, 4.5)])

[('100010110011110110011000011100110000', 0.15029431701487947),
 ('100010110011110110011000011100110000', 0.15029431701487947),
 ('100010110011110110011000011100110000', 0.15029431701487947),
 ('100010110011110110011000011100110000', 0.15029431701487947),
 ('110000100011001001100110010010011010', 0.011398346494599433)]

In [24]:
def k_point_crossover(k_points, p1, p2):
    o1, o2 = '', ''
    index_closer = 0
    for i,point in enumerate(k_points+[len(p1)]):
        if i%2 == 0:
            o1 += p1[index_closer:point]
            o2 += p2[index_closer:point]
        else:
            o1 += p2[index_closer:point]
            o2 += p1[index_closer:point]
        index_closer = point
    return o1,o2

In [26]:
k_point_crossover([2,17],'1000101100hola0110011000011100110000',
 '11010100001011adios10100100101110110')

('10010100001011adi0011000011100110000',
 '1100101100hola011os10100100101110110')

In [25]:
def grouped(iterable, n):
    "s -> (s0,s1,s2,...sn-1), (sn,sn+1,sn+2,...s2n-1), (s2n,s2n+1,s2n+2,...s3n-1), ..."
    return zip(*[iter(iterable)]*n)

In [26]:
for i,j in grouped([3,2,7,4,10,9,0,14],2):
    print(i,j)

3 2
7 4
10 9
0 14


In [27]:
def mutate(individual, pm):
    mutated = ''
    for i in individual:
        if random() < pm:
            if i == '0':
                mutated += '1'
            else:
                mutated += '0'
        else:
            mutated += i
    return mutated        

In [28]:
mutate('1111111111',.1)

'0111111011'

In [63]:
def mutate_pop(population, pm, n_variable_bits_req, n_variable_bounds, fitness_func=Beale_fitness):
    mutated_pop = []
    length = len(population[0])
    for i in range(len(population)):
        mutated_ind = mutate(population[i][0],pm)
        mutated_pop.append((mutated_ind, fitness_func(mutated_ind, n_variable_bits_req, n_variable_bounds)))     
    return mutated_pop

In [64]:
mutate_pop([('11111111111111111',.4),('00000000000000000',.1)],.1,
          bits_req([(-4.5, 4.5),(-4.5, 4.5)]),
                               [(-4.5, 4.5),(-4.5, 4.5)])

[('11111110111111111', 5.731734830878345e-06),
 ('00001001000100000', 6.363818681642189e-06)]

In [130]:
def GA(μ, gens, pc, pm, n_variable_bounds, fitness_func=Beale_fitness):
    n_variable_bits_req = bits_req(n_variable_bounds)
    total_bits = sum(n_variable_bits_req)
    population = [random_solution(n_variable_bits_req,n_variable_bounds,fitness_func=fitness_func) for i in range(μ)]
    for gen in range(gens):
        parents = universal_stochastic_selection(population, n_variable_bits_req, n_variable_bounds)
        offspring = []
        for p1,p2 in grouped(parents,2):
            offs = []
            if random() < pc:
                cross_points = sorted(sample(list(range(total_bits)),2))
                o1,o2 = k_point_crossover(cross_points,p1[0],p2[0])
                o1 = (o1, fitness_func(o1, n_variable_bits_req, n_variable_bounds))
                o2 = (o2, fitness_func(o2, n_variable_bits_req, n_variable_bounds))
                offs = [o1,o2]
            else:
                offs = [p1,p2]
            offspring += offs
        
        offspring = mutate_pop(offspring,pm,n_variable_bits_req,n_variable_bounds,fitness_func=fitness_func)
        offspring.sort(key=lambda x:x[1], reverse=True)
        parents.sort(key=lambda x:x[1], reverse=True)
        
        if parents[0][1] > offspring[0][1]:
            offspring[-1] = parents[0]
            offspring.sort(key=lambda x:x[1], reverse=True)
        population = offspring
    population.sort(key=lambda x:x[1], reverse=True)
    return population[0]

In [131]:
GA(100, 1, 0.9, 0.001,[(-4.5,4.5),(-4.5,4.5)])

('1110000110011010001111010101', 0.898814041807837)

In [109]:
GA(100, 2000, 0.9, 0.001,[(-4.5,4.5),(-4.5,4.5)])

('1101010101010110001110001110', 0.9999996826798102)

In [110]:
real_from_bits('1101010101010110001110001110', n_variable_bits, [(-4.5,4.5),(-4.5,4.5)])

[3.0002746749679545, 0.500183116645303]

## 2.
Utilizar el AG del punto anterior pero ahora con representación real, cruza intermedia completa y mutación uniforme.

In [137]:
def r_Beale_fitness(x): # a low value is a bad fitness
    return 1/(f(*x)+1)

In [138]:
def r_random_solution(n_variable_bounds, fitness_func=r_Beale_fitness):
    individual = []
    for (ul,ll) in n_variable_bounds:
        x_i = uniform(ul,ll)
        individual.append(x_i)
    return individual, fitness_func(individual)

In [139]:
r_random_solution([(-4.5,4.5),(-4.5,4.5)])

([-1.8878485485573546, 1.7234467971926666], 0.033655411425869634)

In [33]:
def r_universal_stochastic_selection(population):
    fitnesses = [x[1] for x in population]
    fitness_mean = mean(fitnesses)
    Ei_s = [fit/fitness_mean for fit in fitnesses]
    r = random()
    parents = []
    sum_ = 0
    for i in range(len(population)):
        sum_ += Ei_s[i]
        while sum_ > r:
            parents.append(population[i])
            r += 1
    return parents

In [146]:
r_universal_stochastic_selection([([2.980120021219805, 2.839468013308001], 0.00019234997626107396),
                                  ([2.772370722423309, 0.37991002354702186], 0.940755880929126),
                                  ([0.6543858622047978, -1.8346766400686958], 0.05040192294120086),
                                  ([3.931393369861617, -3.9888203944768246], 1.4938580133960688e-05)])

[([2.772370722423309, 0.37991002354702186], 0.940755880929126),
 ([2.772370722423309, 0.37991002354702186], 0.940755880929126),
 ([2.772370722423309, 0.37991002354702186], 0.940755880929126),
 ([0.6543858622047978, -1.8346766400686958], 0.05040192294120086)]

In [82]:
def complete_intermediate_cross(p1,p2):
    p1,p2 = np.array(p1), np.array(p2)
    alpha = random()
    o1 = alpha*p1 + (1-alpha)*p2
    o2 = alpha*p2 + (1-alpha)*p1
    return (list(o1),0),(list(o2),0)#r_Beale_fitness(*list(o1))),(list(o2),r_Beale_fitness(*list(o2)))
    # fitnesses are going to be calculated later

In [83]:
complete_intermediate_cross([2.980120021219805, 2.839468013308001],[2.772370722423309, 0.37991002354702186])

(([2.84576611673199, 1.248843057397621], 0),
 ([2.906724626911124, 1.9705349794574019], 0))

In [142]:
def r_mutate(individual, pm, n_variable_bounds, fitness_func=r_Beale_fitness):
    mutated = []
    for var,bounds in zip(individual[0],n_variable_bounds):
        l_inf,l_sup = bounds
        if random() < pm:
            x_i = uniform(l_inf,l_sup)
            mutated.append(x_i)
        else:
            mutated.append(var)
    return mutated, fitness_func(mutated)        

In [87]:
r_mutate(([2.843927530895764, 1.2270759154166462], 0.022439376995842773),.1,[(-4.5,4.5),(-4.5,4.5)])

([2.843927530895764, 1.2270759154166462], 0.022439376995842773)

In [91]:
def r_mutate_pop(population, pm, n_variable_bounds, fitness_func=r_Beale_fitness):
    mutated_pop = []
    length = len(population[0])
    for i in range(len(population)):
        mutated_ind = r_mutate(population[i], pm, n_variable_bounds, fitness_func=fitness_func)
        mutated_pop.append(mutated_ind)     
    return mutated_pop

In [92]:
r_mutate_pop([([2.772370722423309, 0.37991002354702186], 0.940755880929126),
 ([2.772370722423309, 0.37991002354702186], 0.940755880929126),
 ([2.772370722423309, 0.37991002354702186], 0.940755880929126),
 ([0.6543858622047978, -1.8346766400686958], 0.05040192294120086)],
            0.3,
             [(-4.5,4.5),(-4.5,4.5)])

[([2.772370722423309, 0.7412868013861029], 0.2794062953880693),
 ([-3.263638793473766, 0.37991002354702186], 0.013996773349670248),
 ([2.772370722423309, 0.11504839209831275], 0.4618147971877947),
 ([0.6543858622047978, -1.8346766400686958], 0.05040192294120086)]

In [143]:
def real_GA(μ, gens, pc, pm, n_variable_bounds, fitness_func=r_Beale_fitness):
    population = [r_random_solution(n_variable_bounds, fitness_func=fitness_func) for i in range(μ)]
    for gen in range(gens):
        parents = r_universal_stochastic_selection(population)
        offspring = []
        for p1,p2 in grouped(parents,2):
            offs = []
            if random() < pc:
                o1,o2 = complete_intermediate_cross(p1[0],p2[0])
                offs = [o1,o2]
            else:
                offs = [p1,p2]
            offspring += offs
        
        offspring = r_mutate_pop(offspring,pm,n_variable_bounds,fitness_func=fitness_func)
        offspring.sort(key=lambda x:x[1], reverse=True)
        parents.sort(key=lambda x:x[1], reverse=True)
        
        if parents[0][1] > offspring[0][1]:
            offspring[-1] = parents[0]
            offspring.sort(key=lambda x:x[1], reverse=True)
        population = offspring
    population.sort(key=lambda x:x[1], reverse=True)
    return population[0]

In [144]:
real_GA(100, 1, 0.9, 0.001,[(-4.5,4.5),(-4.5,4.5)])

([1.5229481891615118, 0.07382242788254507], 0.3617136979855048)

In [266]:
real_GA(100, 2000, 0.9, 0.001,[(-4.5,4.5),(-4.5,4.5)])

([2.9992362901614458, 0.49961453461013206], 0.9999990215327584)

## 3.
Utiliza las dos versiones del AG que se tienen para resolver la función de Ackley.

In [98]:
def Ackley(x):
    n = len(x)
    return -20*e**(-0.2*((1/n)*sum(x_i**2 for x_i in x))) - e**((1/n)*sum(cos(2*pi*x_i) for x_i in x)) + 20 + e     

In [105]:
Ackley([3,2,-2,-7])

19.2623366519752

In [173]:
def Ackley_fitness(x):
    return 1/(Ackley(x)+1)

In [146]:
def Ackley_fitness_bin(bit_string, n_variable_bits_req, n_variable_bounds):
    return 1/(Ackley(real_from_bits(bit_string, n_variable_bits_req, n_variable_bounds))+1)
#-Ackley(real_from_bits(bit_string, n_variable_bits_req, n_variable_bounds))

In [147]:
var_bounds = [(-30,30),(-30,30)]
n_variable_bits = bits_req(var_bounds)
n_variable_bits

[16, 16]

In [148]:
rand_string = random_bin_string(32)
rand_string

'01101011101100000010100011001110'

In [154]:
Ackley_fitness_bin(rand_string, n_variable_bits, var_bounds)

0.04335307451919739

In [179]:
GA(100, 1, 0.9, 0.001,[(-30,30),(-30,30)], fitness_func=Ackley_fitness_bin)

('01111001010100010111101110110011', 0.11714994137541734)

In [180]:
GA(100, 1, 0.9, 0.001,[(-30,30),(-30,30),(-30,30),(-30,30)], fitness_func=Ackley_fitness_bin)

('1000010010100100011110100111111000100110011100100110111011101010',
 0.12484084361687128)

In [182]:
real_GA(100, 1, 0.9, 0.001,[(-30,30),(-30,30),(-30,30),(-30,30)], fitness_func=Ackley_fitness)

([-3.7095153803258087,
  4.9815370552848055,
  -0.5415347816529756,
  -3.569236373485266],
 0.04662040046939444)

In [191]:
GA(100, 2000, 0.9, 0.001,[(-30,30),(-30,30),(-30,30),(-30,30)], fitness_func=Ackley_fitness_bin)

('1000000000000000100000000000000010100010100110110100111110101101',
 0.9999879179795412)

In [192]:
real_GA(100, 2000, 0.9, 0.001,[(-30,30),(-30,30),(-30,30),(-30,30)], fitness_func=Ackley_fitness)

([0.00045633520433435585,
  0.002370489336074353,
  -0.00011056845972334356,
  0.0006739139146811777],
 0.9999092902841624)

## 4.
Realizar una tabla comparativa que reporte los resultados de ambas versiones en los dos problemas de prueba (función de Beale y función de Ackley). En el caso de la función de Ackley, resolver instancias con 5, 10 y 20 variables. Para construir la tabla comparativa debes realizar M ejecuciones de cada versión resolviendo cada uno de los problemas y reportar lo siguiente:
1. Mejor solución encontrada considerando las M ejecuciones.
2. Peor solución encontrada considerando las M ejecuciones.
3. Solución que corresponde a la mediana considerando las M ejecuciones.
4. Media del valor de la función objetivo considerando las M ejecuciones.
5. Desviación estándar del valor de la función objetivo considerando las M ejecuciones.

En los primeros tres puntos indica tanto el valor de x como el valor de la función objetivo f. Nota: recuerde que puede variar los parámetros pc, pm y tamaño de la población para obtener mejores resultados. Indique los valores que utilizó para cada problema.